# Preparation

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

# Q1. Define function description

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather", #TODO 1
    "description": "Retrieve the weather temperature for a specified city.", #TODO 2
    "parameters": {
        "type": "object",
        "properties": {
            "city": { # TODO 3
                "type": "string",
                "description": "The name of the city to get weather information for." #TODO 4
            }
        },
        "required": ["city"], #TODO 5
        "additionalProperties": False
    }
}

# Q2. Adding another tool

In [3]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [4]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the weather temperature for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to set the weather for."
            },
            "temp": {
                "type": "number",
                "description": "The temperature to set for the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

# Q3. Install FastMCP

In [5]:
import fastmcp

print(fastmcp.__version__)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
2.10.4


# Q4. Simple MCP Server

In [9]:
# weather_server.py
from fastmcp import FastMCP, Client
import random

# Simulasi data suhu
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

# Init MCP
mcp = FastMCP("Demo 🚀")

# Daftarkan kedua fungsi sebagai tools
mcp.tool(get_weather)
mcp.tool(set_weather)

import asyncio

async def main():
    async with Client(mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print("Available tools:", tools)

await main()

Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem

# Q5. Protocol

In [11]:
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": "true"}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}

{'jsonrpc': '2.0',
 'id': 1,
 'method': 'initialize',
 'params': {'protocolVersion': '2024-11-05',
  'capabilities': {'roots': {'listChanged': 'true'}, 'sampling': {}},
  'clientInfo': {'name': 'test-client', 'version': '1.0.0'}}}

In [12]:
{"jsonrpc": "2.0", "method": "notifications/initialized"}

{'jsonrpc': '2.0', 'method': 'notifications/initialized'}

In [13]:
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}

{'jsonrpc': '2.0', 'id': 2, 'method': 'tools/list'}

In [17]:
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "get_weather",
    "arguments": {
      "city": "Berlin"
    }
  }
}

{'jsonrpc': '2.0',
 'id': 3,
 'method': 'tools/call',
 'params': {'name': 'get_weather', 'arguments': {'city': 'Berlin'}}}